# DDL e DML

O SQL usa os termos **tabela**, **linha** e **coluna** para os termos do modelo relacional formal *relação*, *tupla* e *atributo* respectivamente.

In [1]:
# Utilizaremos a lib csvms em nossos exemplos
from csvms.table import Table

## Criação

O comando `CREATE TABLE` é usado para especificar uma nova relação, dando-lhe um **nome** e especificando seus **atributos**. Cada um dos atributos recebe um **nome** e um **tipo de dado** para especificar seu domínio de valores.

```sql
CREATE TABLE lista_frutas (
    nm_fruta TEXT ,
    tp_fruta TEXT )
```

> Como nosso banco de dados será escrito na linguagem [python](https://www.w3schools.com/python/default.asp), utilizaremos os seus [tipos primitivos](https://www.w3schools.com/python/python_datatypes.asp) como tipos de dados.

Os tipos de dados básicos disponíveis para atributos costumam ser:

+ Numéricos
+ Cadeia de caracters
+ Cadeia de bits
+ Booleano
+ Data
+ Hora

> E é já a partir deste ponto em que os vários dialetos começam a surgir...

![create table](../imgs/create_table.png)

In [3]:
# Criação da tabela
Table(
    name="lista_frutas",
    columns={
        'nm_fruta':str,
        'tp_fruta':str
    }
).save()

True

## Transação

O inicio do trabalho `BEGIN` se dá com o carregamento da tabela do sistema de arquivos para a SGA (System Global Area).
Tudo o que ocorrer com este objeto durante seu período em mória será considerado uma transação até que se confirme com os comandos de `COMMIT` ou `ROLLBACK`

In [11]:
# Carregar os dados da tabela para a memória
tbl = Table('default.lista_frutas')

In [7]:
print(tbl)

TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
   +---------+---------+



## Inclusão

Em sua forma mais simples, o comando `INSERT` é usado para acrescentar uma única *tupla* (linha) em uma *relação* (tabela).
Precisamos especificar o nome da relação e uma lista de valores para a tupla. Os valores devem ser informados na mesma ordem em que os atributos correspondentes foram especificados no `CREATE TABLE`.

![insert](../imgs/insert.png)

```sql
INSERT INTO lista_frutas VALUES ('banana','doce');
INSERT INTO lista_frutas VALUES ('limão','azedo');
INSERT INTO lista_frutas VALUES ('maçã','doce');
```

In [8]:
# Inserir as tres linhas conforme descrito
tbl.append('banana','doce')
tbl.append('limão','azedo')
tbl.append('maçã','doce')

2022-07-25 21:45:00,443: Row ('banana', 'doce') inserted
2022-07-25 21:45:00,445: Row ('limão', 'azedo') inserted
2022-07-25 21:45:00,446: Row ('maçã', 'doce') inserted


True

In [9]:
print(tbl)

TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



## Confirmação

Com o comando de `COMMIT` confirmamos que todas as alterações desta transação deverão ser **salvas** em definitivo.

```sql
COMMIT
```

In [10]:
# Salvar os dados da tabela no sistema de arquivos
tbl.save()

True

## Atualização

O comando `UPDATE` é utilizado para modificar valores de atributos de uma ou mais *tuplas* selecionadas. Assim como no comando `DELETE`, é possível adicionar uma clausula `WHERE` no comando para informar o critério de seleção das linhas que serão afetadas.

```sql
UPDATE lista_frutas 
   SET tp_fruta = 'amargo'
 WHERE tp_fruta = 'doce'
```

In [14]:
# Alterar todas as frutas do tipo 'doce' para 'amargo'
for idx, row in enumerate(tbl):
    if row[1]=='doce':
        new = list(row)
        new[1]='amargo'
        tbl[idx]=new

print(tbl)

2022-07-25 21:52:24,807: Row 0 updated with values ['banana', 'amargo']
2022-07-25 21:52:24,809: Row 2 updated with values ['maçã', 'amargo']


TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|   amargo|
  1|    limão|    azedo|
  2|     maçã|   amargo|
   +---------+---------+



## Exclusão

O comando `DELETE` remove *tuplas* de uma relação. Ele possui uma clausula *`WHERE`*, semelhante a utilzada em consultas, para selecionar a tuplas que serão excluídas.

```sql
DELETE FROM lista_frutas WHERE nm_fruta = 'limão'
```

In [15]:
# Procura em toda a tabela por linhas cujo nome da fruta seja "limão"
for idx, row in enumerate(tbl):
    if row[0]=='limão':
        del tbl[idx]

print(tbl)


2022-07-25 21:54:39,025: Row ('limão', 'azedo') deleted


TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|   amargo|
  1|     maçã|   amargo|
   +---------+---------+



## Descarte

Com o comando de `ROLLBACK` descartamos todas as alterações desta transação, voltando assim ao estado em que os dados se encontravam desde a ultima operação salva. O comando de `ROLLBACK` finalizará um trabalho e iniciará outro.

```sql
ROLLBACK
```

In [16]:
# Descartar todas as modificações realizadas
tbl = Table('lista_frutas')

In [17]:
print(tbl)

TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



## Alteração

O comando `ALTER TABLE` serve para alterar os atributos definidos para a tabela durante o processo de criação `CREATE TABLE`. Seus principais operadores servem para adição (`ADD`), modificação (`MODIFY`) tipo de dado, renomear (`RENAME`) um atributo ou ainda a remoção (`DROP`) de atributos existentes.

> Este sistema não prevê *concorrencia*, por isso podemos trabalhar sobre os objetos da SGA

```sql
ALTER TABLE lista_frutas ADD (valor INTEGER)
```

In [9]:
# Adiciona uma nova coluna do tipo inteiro chamada valor

```sql 
ALTER TABLE lista_frutas MODIFY (valor INTEGER) TO (vl_fruta FLOAT);
```

In [10]:
# Renomear a coluna para a convenção de nomes utilizada na tabela e alterar seu tipo para ponto flutuante

```sql
ALTER TABLE lista_frutas DROP vl_fruta;
```

In [11]:
# Remove a coluna 'vl_fruta' da tabela

## Eliminação

Como o nome diz, o comando `DROP TABLE` eliminará o objeto do sistema. Ou seja, tanto o que está em memória quanto no sistema de arquivos e catalogo de dados.

![drop table](../imgs/drop_table.png)

In [12]:
# Eliminar a tabela do sistema